In [9]:
from bokeh.plotting import figure, output_notebook, show
import math
from bokeh.models.widgets import Select
from bokeh.models import CustomJS, ColumnDataSource, DataRange1d, Plot, BasicTicker, LinearAxis, PanTool, WheelZoomTool, Grid, Range1d
from bokeh.models.glyphs import Circle
from bokeh.layouts import column
import itertools
from bokeh.layouts import gridplot
from re import match
from scipy import stats
import numpy as np

#self-defined module
import data_parser

output_notebook()
#import excel datasets, and convert them into dataframe
wholesale_data_missing_df = data_parser.parse_csv_data('./data/Wholesale_customers_data-missing.csv')
wholesale_names = data_parser.extract_column_names(wholesale_data_missing_df)
#Extract missing data
data_missing_rows = data_parser.extract_data_missing_rows(wholesale_data_missing_df)
data_missing_rows_indices = [r['row'] for r in data_missing_rows]
print("Print out the indices of rows and columns of missing data:")
print(data_missing_rows)
wholesale_data_drop_missing_df = data_parser.drop_rows_from_df(wholesale_data_missing_df, data_missing_rows_indices)
wholesale_data_drop_missing_dic = data_parser.build_dic(wholesale_data_drop_missing_df, wholesale_names)

#The following codes for generating the grid of plots are from https://bokeh.pydata.org/en/latest/docs/gallery/iris_splom.html#iris-splom-py
missing_data_plots = []
xattrs = wholesale_names
yattrs = list(reversed(xattrs))

xdr = DataRange1d(bounds=None)
ydr = DataRange1d(bounds=None)

source = ColumnDataSource(
    data=wholesale_data_drop_missing_dic
)


def make_plot(xname, yname, xax=False, yax=False):
    mbl = 40 if yax else 0
    mbb = 40 if xax else 0
    plot = Plot(
        x_range=xdr, y_range=ydr, background_fill_color="#efe8e2",
        border_fill_color='white', plot_width=200 + mbl, plot_height=200 + mbb,
        min_border_left=2+mbl, min_border_right=2, min_border_top=2, min_border_bottom=2+mbb)
    circle = Circle(x=xname, y=yname, fill_alpha=0.2, size=4)
    r = plot.add_glyph(source, circle)
    xdr.renderers.append(r)
    ydr.renderers.append(r)

    xticker = BasicTicker()
    if xax:
        xaxis = LinearAxis(major_label_orientation = 'vertical', axis_label = xname)
        plot.add_layout(xaxis, 'below')
        xticker = xaxis.ticker
    plot.add_layout(Grid(dimension=0, ticker=xticker))

    yticker = BasicTicker()
    if yax:
        yaxis = LinearAxis(axis_label = yname)
        plot.add_layout(yaxis, 'left')
        yticker = yaxis.ticker
    plot.add_layout(Grid(dimension=1, ticker=yticker))

    plot.add_tools(PanTool(), WheelZoomTool())

    return plot
    

for y in yattrs:
    row = []
    for x in xattrs:
        xax = (y == yattrs[-1])
        yax = (x == xattrs[0])
        row.append(make_plot(x, y, xax, yax))
    missing_data_plots.append(row)
grid = gridplot(missing_data_plots)
show(grid)




Loading BokehJS ...

Print out the indices of rows and columns of missing data:
[{'row': 75, 'col': [4]}, {'row': 172, 'col': [4]}, {'row': 180, 'col': [3]}, {'row': 225, 'col': [6]}, {'row': 274, 'col': [7]}, {'row': 370, 'col': [6]}]


## Solution1: Remove the missing data rows from the datasets.
In the plots above, the rows of data with missing datum are removed from the dataframe. Since the size of datasets is big, it does not affect much without those 6 rows of data. 
What's more, as we can see from the plots above, Grocery and Detergents Paper tend to be linear regression. Therefore, we can generate two least-squares regressions to predict the missing data of grocery and detergents_paper.

In [3]:
x_linear = wholesale_data_drop_missing_dic['Grocery'] #index 4
y_linear = wholesale_data_drop_missing_dic['Detergents_Paper'] #index 6
#Get least-squares regression 
slope, intercept, r_value, p_value, std_err = stats.linregress(x_linear, y_linear)

x_linear0 = wholesale_data_missing_df.iloc[225, :]['Grocery']
pre_y_linear_missing0 = intercept + slope * x_linear0
print("Predicted value of Detergents_paper: ", pre_y_linear_missing0, " Actual value of Detergents_paper: 761")
x_linear1 = wholesale_data_missing_df.iloc[370, :]['Grocery']
pre_y_linear_missing1 = intercept + slope * x_linear1
print("Predicted value of Detergents_paper: ", pre_y_linear_missing1, " Actual value of Detergents_paper: 523")

x_linear_deter = wholesale_data_drop_missing_dic['Detergents_Paper']
y_linear_gro = wholesale_data_drop_missing_dic['Grocery']
#Get least-squares regression 
slope_deter_gro, intercept_deter_gro, r_value_deter_gro, p_value_deter_gro, std_err_deter_gro = stats.linregress(x_linear_deter, y_linear_gro)

#y_linear = intercept + slope * x_linear
x_linear_deter0 = wholesale_data_missing_df.iloc[75, :]['Detergents_Paper']
pre_y_linear_gro_missing0 = intercept_deter_gro + slope_deter_gro * x_linear_deter0
print("Predicted value of Grocery: ", pre_y_linear_gro_missing0, " Actual value of Grocery: 1137")
x_linear_deter1 = wholesale_data_missing_df.iloc[172, :]['Detergents_Paper']
pre_y_linear_gro_missing1 = intercept_deter_gro + slope_deter_gro * x_linear_deter1
print("Predicted value of Grocery: ", pre_y_linear_gro_missing1, " Actual value of Grocery: 5479")



Predicted value of Detergents_paper:  1123.02124384  Actual value of Detergents_paper: 761
Predicted value of Detergents_paper:  1493.40083707  Actual value of Detergents_paper: 523
Predicted value of Grocery:  2649.6329707  Actual value of Grocery: 1137
Predicted value of Grocery:  7877.17350612  Actual value of Grocery: 5479


## Solution 2: Find the Linear Regression between Grocery and Detergents Paper

In [4]:
source_gro_deter = ColumnDataSource(data=dict(x=wholesale_data_drop_missing_dic['Grocery'], 
                                              y=wholesale_data_drop_missing_dic['Detergents_Paper'], 
                                              color=["navy"]*len(wholesale_data_drop_missing_dic['Grocery'])))
source_gro_deter_linear = ColumnDataSource(data=dict(x=wholesale_data_drop_missing_dic['Grocery'], 
                                                     y=[intercept + slope * x for x in wholesale_data_drop_missing_dic['Grocery']]))
#Get the minium and maximum value among grocery data
grocery_min = min(wholesale_data_drop_missing_dic['Grocery'])
grocery_max = max(wholesale_data_drop_missing_dic['Grocery'])
#Get the mean value among detergents_paper data
mean_detergents_paper = np.average(wholesale_data_drop_missing_dic['Detergents_Paper'])
source_gro_deter_avg = ColumnDataSource(data=dict(xm=[grocery_min, grocery_max],
                                                  ym=[mean_detergents_paper, mean_detergents_paper]))

p_gro_deter = figure(plot_width=400, plot_height=400, tools="lasso_select,wheel_zoom,pan")
p_gro_deter.circle('x', 'y', color='color', source = source_gro_deter, size=5, alpha=0.5)
p_gro_deter.line('x', 'y', source = source_gro_deter_linear, line_color="orange", line_dash="4 4")
p_gro_deter.line(x='xm', y='ym', color="green", line_width=5, alpha=0.6, source=source_gro_deter_avg)
p_gro_deter.title.text = 'The Linear regression between Grocery and Detergents_Paper'
p_gro_deter.xaxis.axis_label = "Grocery"
p_gro_deter.yaxis.axis_label = "Detergents_Paper"
source_gro_deter.callback = CustomJS(args=dict(s2=source_gro_deter_avg), code="""
    var inds = cb_obj.get('selected')['1d'].indices;
    var d = cb_obj.get('data');
    var ym = 0
    
    if (inds.length == 0) { return; }
    
    for (i = 0; i < d['color'].length; i++) {
        d['color'][i] = "navy"
    }
    for (i = 0; i < inds.length; i++) {
        d['color'][inds[i]] = "firebrick"
        ym += d['y'][inds[i]]
    }
    
    ym /= inds.length
    s2.get('data')['ym'] = [ym, ym]
    
    cb_obj.trigger('change');
    s2.trigger('change');
""")
show(p_gro_deter)

The least-square regression model might be affected by noisy.

In [7]:
#Generate the plot of Milk and Detergents_paper.
# sum_gro_deter_del = [x + y + z for x, y, z in zip(wholesale_data_drop_missing_dic['Grocery'], 
#                                                  wholesale_data_drop_missing_dic['Detergents_Paper'], 
#                                                  wholesale_data_drop_missing_dic['Delicassen'])]

p_del_milk = figure(plot_width=400, plot_height=400, tools="lasso_select,wheel_zoom,pan")
source_del_milk = ColumnDataSource(data=dict(x=wholesale_data_drop_missing_dic['Delicassen'], 
                                              y=wholesale_data_drop_missing_dic['Milk'], 
                                              color=["navy"]*len(wholesale_data_drop_missing_dic['Milk'])))
p_del_milk.xaxis.axis_label = "Delicassen"
p_del_milk.yaxis.axis_label = "Milk"
p_del_milk.title.text = "Plot of Delicassen and Milk"
#p_sum_milk.y_range = Range1d(0, 15)
p_del_milk.circle('x', 'y', color='color', source = source_del_milk, size=5, alpha=0.5)
del_min = min(wholesale_data_drop_missing_dic['Delicassen'])
del_max = max(wholesale_data_drop_missing_dic['Delicassen'])
mean_del = np.average(wholesale_data_drop_missing_dic['Delicassen'])
print("Mean of Delicassen: ", mean_del)

milk_min = min(wholesale_data_drop_missing_dic['Milk'])
milk_max = max(wholesale_data_drop_missing_dic['Milk'])
mean_milk = np.average(wholesale_data_drop_missing_dic['Milk'])
print("Mean of Milk: ", mean_milk)

source_milk_avg = ColumnDataSource(data=dict(xm=[del_min, del_max],
                                                  ym=[mean_milk, mean_milk]))
source_del_avg = ColumnDataSource(data=dict(xm=[mean_del, mean_del],
                                                  ym=[del_min, del_max]))
p_del_milk.line(x='xm', y='ym', color="green", line_width=5, alpha=0.6, source=source_milk_avg, legend="mean of Milk")
p_del_milk.line(x='xm', y='ym', color="red", line_width=5, alpha=0.6, source=source_del_avg, legend="mean of Delicassen")
source_del_milk.callback = CustomJS(args=dict(s2=source_milk_avg, s3=source_del_avg), code="""
    var inds = cb_obj.get('selected')['1d'].indices;
    var d = cb_obj.get('data');
    var ym = 0
    var xm = 0
    
    if (inds.length == 0) { return; }
    
    for (i = 0; i < d['color'].length; i++) {
        d['color'][i] = "navy"
    }
    for (i = 0; i < inds.length; i++) {
        d['color'][inds[i]] = "firebrick"
        ym += d['y'][inds[i]]
        xm += d['x'][inds[i]]
    }
    ym /= inds.length
    xm /= inds.length
    s2.get('data')['ym'] = [ym, ym]
    s3.get('data')['xm'] = [xm, xm]
    
    cb_obj.trigger('change');
    s2.trigger('change');
    s3.trigger('change');
""")
show(p_del_milk)

Mean of Delicassen:  1526.02073733
Mean of Milk:  5826.76267281


## Solution 3: Use attributes means to implement the missing data
The actual value for milk: 6036, the mean value for milk: 5826
The actual value for delicassen: 1153, the mean value for delicassen: 1526

As we can see from the result above, implementing the mean value might be a better solution than generating linear regressions.